In [31]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
experiment = Experiment(ws, 'Bank_Marketing_AutoML')
print("Experiment created")

print('WORKSPACE DETAILS:\nWorkspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Experiment created
WORKSPACE DETAILS:
Workspace name: quick-starts-ws-134051
Azure region: southcentralus
Subscription id: 5a4ab2ba-6c51-4805-8155-58759ad589d8
Resource group: aml-quickstarts-134051


In [32]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
### YOUR CODE HERE ###

cpu_cluster_name = "cpu-cluster"

# to check whether the compute cluster exists already or not
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Existing compute target found... Using it")

except ComputeTargetException:
    print("Creating new Compute Target... " + cpu_cluster_name)
    provisioning_compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, provisioning_compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Existing compute target found... Using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [33]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url_path)

KeyboardInterrupt: 

In [ ]:
# checking the uploaded data
ds_dataframe = ds.to_pandas_dataframe()
ds_dataframe.head()

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x.head()

In [ ]:
# saving cleaned data locally

import os
import pandas as pd
path = "./Data"
try:
    os.makedirs(path, exist_ok=True)
    print("Dicrectory '%s' created...")
except OSError:
    print("Directory '%s' cannot be created...")


In [ ]:
clean_df = x.copy(deep=True)
clean_df['y'] = y
clean_df.to_csv('./Data/clean_data.csv')
print(x.shape, y.shape, clean_df.shape)
print(clean_df.head())

In [ ]:
# split data into test and train sets

from sklearn.model_selection import train_test_split
import pandas as pd

#spliting the data
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3)

# copying series y column to the train data
train_df = train_x.copy(deep=True)
train_df['y'] = train_y
print(train_x.shape, train_y.shape)

# joining series y column to the test data
test_df = test_x.copy(deep=True)
test_df['y'] = test_y
print(test_x.shape, test_y.shape)

print(train_df.shape)
print(test_df.shape)

# saving test and train data locally
train_df.to_csv("./Data/train_data.csv")
test_df.to_csv("./Data/test_data.csv")

In [ ]:
# upload the Data directory to the datastore

deafult_store = ws.get_default_datastore()
deafult_store.upload(src_dir="./Data", target_path="./Data", overwrite=True)

In [ ]:
# Convert the DataFrame to TabularDataset
from azureml.core import Dataset

clean_data = Dataset.Tabular.from_delimited_files(path=[(deafult_store, ("./Data/clean_data.csv"))])
train_data = Dataset.Tabular.from_delimited_files(path=[(deafult_store, ("./Data/train_data.csv"))])
test_data = Dataset.Tabular.from_delimited_files(path=[(deafult_store, ("./Data/test_data.csv"))])

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name="y",
    n_cross_validations=5,
    compute_target = cpu_cluster)

print("AutoML config created,.")

In [ ]:
# Submit your automl run
from azureml.widgets import RunDetails

### YOUR CODE HERE ###
automl_run = experiment.submit(automl_config, show_output = True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

In [ ]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
best_run.register_model(model_name="Bank_Marketing_AutoML_best_model.pkl", model_path ="./outputs/")

In [ ]:
best_run.get_tags()

In [ ]:
best_run